In [ ]:
from semantic_search_to_json import Output_Analysis

Output_Analysis("How to play monopoly" , 'input_small.json')


In [6]:
from llm4_to_json import extract_outline_and_title
import pymupdf4llm
import json
import os

def parse_pdf_for_text_and_outline(docs_path=None):
    all_outlines = []
    text_and_meta = []
    docs_path = "/home/lord_grivous/Desktop/1Breal/AntonB/data/"
    for file in [file for file in os.listdir(docs_path) if file.endswith('.pdf')]:
        file_path = os.path.join(docs_path, file)

        md_text = pymupdf4llm.to_markdown(file_path, page_chunks=True)
        all_outlines.append(extract_outline_and_title(md_text))

        page_text = ""
        for pages in md_text:        
            page_text += pages["text"]
        text_and_meta.append({
            "text": page_text,
            "document": file_path,
        })

    return text_and_meta, all_outlines
    


In [7]:
text_and_meta, all_outlines =  parse_pdf_for_text_and_outline()


In [8]:
from fastembed.embedding import DefaultEmbedding
import numpy as np

# Load small embedding model (auto-downloads)
embedder = DefaultEmbedding(model_name="BAAI/bge-small-en-v1.5", cache_dir="emb_models")

def avg_embed(text: str) -> np.ndarray:
    embeddings = list(embedder.embed(text))
    return np.array(embeddings[0]) if embeddings else np.zeros(embedder.embedding_size)


In [9]:
avg_embeddings = [avg_embed(item["text"]) for item in text_and_meta]